In [20]:
import pandas as pd
import yfinance as yf
from yahoo_fin import stock_info as si
from sqlalchemy import create_engine
import urllib
import time
from datetime import datetime
from datetime import date
import datetime

#### **Отримуємо список тікерів, що входять до індексу SP500**

In [6]:
df_SP500 = pd.DataFrame( si.tickers_sp500() )
df_SP500

,0
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
498,YUM
499,ZBH
500,ZBRA
501,ZION


In [7]:
ticker_list = df_SP500[df_SP500.columns[0]].values.tolist()
len(ticker_list)

503

In [8]:
ticker_list[0:5]

['A', 'AAL', 'AAP', 'AAPL', 'ABBV']

In [9]:
pd.options.display.max_colwidth = 5000

#### **Отримуємо інформацію про компанію та дані по виплатах дивідендів для всіх акцій з SP500**

In [10]:
df_stock_final = pd.DataFrame()
df_dividend_final = pd.DataFrame()
i = 1

for ticker in ticker_list:
  stock = yf.Ticker(ticker)

  keys_to_extract = ['city','state','zip','website','industry','sector','longBusinessSummary',
                   'marketCap','shortName','longName','symbol','underlyingSymbol']
  stock_filtered = dict(filter(lambda item: item[0] in keys_to_extract, stock.info.items()))
  df_stock = pd.DataFrame(stock_filtered, index=[0])
  df_stock_final=pd.concat([df_stock_final, df_stock], ignore_index=True)

  df_dividends = stock.dividends.to_frame().reset_index()

  if df_dividends.shape[0] > 0:
    df_dividends['Ticker'] = ticker
    df_dividend_final=pd.concat([df_dividend_final, df_dividends], ignore_index=True)
  
  print(str(i) + '. '+ ticker + '. Stock data (rows): ' + str(df_stock.shape[0]) + '. Dividend data (rows): ' + str(df_dividends.shape[0]) + '.')
  i += 1

1. A. Stock data (rows): 1. Dividend data (rows): 46.
2. AAL. Stock data (rows): 1. Dividend data (rows): 23.
3. AAP. Stock data (rows): 1. Dividend data (rows): 69.
4. AAPL. Stock data (rows): 1. Dividend data (rows): 78.
5. ABBV. Stock data (rows): 1. Dividend data (rows): 42.
6. ABC. Stock data (rows): 1. Dividend data (rows): 87.
7. ABT. Stock data (rows): 1. Dividend data (rows): 162.
8. ACGL. Stock data (rows): 1. Dividend data (rows): 0.
9. ACN. Stock data (rows): 1. Dividend data (rows): 39.
10. ADBE. Stock data (rows): 1. Dividend data (rows): 63.
11. ADI. Stock data (rows): 1. Dividend data (rows): 78.
12. ADM. Stock data (rows): 1. Dividend data (rows): 160.
13. ADP. Stock data (rows): 1. Dividend data (rows): 160.
14. ADSK. Stock data (rows): 1. Dividend data (rows): 63.
15. AEE. Stock data (rows): 1. Dividend data (rows): 101.
16. AEP. Stock data (rows): 1. Dividend data (rows): 240.
17. AES. Stock data (rows): 1. Dividend data (rows): 48.
18. AFL. Stock data (rows): 1. Di

In [11]:
df_stock_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   city                 502 non-null    object
 1   state                481 non-null    object
 2   zip                  501 non-null    object
 3   website              502 non-null    object
 4   industry             502 non-null    object
 5   sector               502 non-null    object
 6   longBusinessSummary  502 non-null    object
 7   marketCap            503 non-null    int64 
 8   symbol               503 non-null    object
 9   underlyingSymbol     503 non-null    object
 10  shortName            503 non-null    object
 11  longName             503 non-null    object
dtypes: int64(1), object(11)
memory usage: 47.3+ KB


In [12]:
df_dividend_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46335 entries, 0 to 46334
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype                           
---  ------     --------------  -----                           
 0   Date       46335 non-null  datetime64[ns, America/New_York]
 1   Dividends  46335 non-null  float64                         
 2   Ticker     46335 non-null  object                          
dtypes: datetime64[ns, America/New_York](1), float64(1), object(1)
memory usage: 1.1+ MB


In [13]:
df_dividend_final

,Date,Dividends,Ticker
0,2006-11-01 00:00:00-05:00,1.471388,A
1,2012-03-30 00:00:00-04:00,0.071531,A
2,2012-06-29 00:00:00-04:00,0.071531,A
3,2012-09-28 00:00:00-04:00,0.071531,A
4,2012-12-27 00:00:00-05:00,0.071531,A
...,...,...,...
46330,2022-04-20 00:00:00-04:00,0.325000,ZTS
46331,2022-07-20 00:00:00-04:00,0.325000,ZTS
46332,2022-10-31 00:00:00-04:00,0.325000,ZTS
46333,2023-01-19 00:00:00-05:00,0.375000,ZTS


In [14]:
df_stock_final.tail(2)

,city,state,zip,website,industry,sector,longBusinessSummary,marketCap,symbol,underlyingSymbol,shortName,longName
501,Salt Lake City,UT,84133-1109,https://www.zionsbancorporation.com,Banks—Regional,Financial Services,"Zions Bancorporation, National Association provides various banking and related services primarily in the states of Arizona, California, Colorado, Idaho, Nevada, New Mexico, Oregon, Texas, Utah, Washington, and Wyoming. The company offers corporate banking services; commercial banking, including a focus on small- and medium-sized businesses; commercial real estate banking services; municipal and public finance services; retail banking, including residential mortgages; trust services; wealth management and private client banking services; and capital markets products and services. The company was formerly known as ZB, National Association and changed its name to Zions Bancorporation, National Association in September 2018. Zions Bancorporation, National Association was founded in 1873 and is headquartered in Salt Lake City, Utah.",3518855936,ZION,ZION,Zions Bancorporation N.A.,"Zions Bancorporation, National Association"
502,Parsippany,NJ,07054,https://www.zoetis.com,Drug Manufacturers—Specialty & Generic,Healthcare,"Zoetis Inc. discovers, develops, manufactures, and commercializes animal health medicines, vaccines, and diagnostic products in the United States and internationally. It commercializes products primarily across species, including livestock, such as cattle, swine, poultry, fish, and sheep and others; and companion animals comprising dogs, cats, and horses. The company also offers parasiticides; vaccines; anti-infectives; other pharmaceutical products; dermatology; and medicated feed additives. In addition, the company provides animal health diagnostics, including point-of-care diagnostic products and laboratory; and other non-pharmaceutical products. It markets its products to veterinarians, livestock producers, and pet owners. The company was founded in 1952 and is headquartered in Parsippany, New Jersey.",86142361600,ZTS,ZTS,Zoetis Inc.,Zoetis Inc.


In [15]:
df_dividend_final['Date'] = pd.to_datetime( df_dividend_final['Date'], errors='coerce',utc=True)

In [16]:
df_dividend_final['Date'] = df_dividend_final['Date'].dt.date

In [17]:
df_dividend_final

,Date,Dividends,Ticker
0,2006-11-01,1.471388,A
1,2012-03-30,0.071531,A
2,2012-06-29,0.071531,A
3,2012-09-28,0.071531,A
4,2012-12-27,0.071531,A
...,...,...,...
46330,2022-04-20,0.325000,ZTS
46331,2022-07-20,0.325000,ZTS
46332,2022-10-31,0.325000,ZTS
46333,2023-01-19,0.375000,ZTS


In [21]:
quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};Server=VIKTOR-PC-2001;Database=DividendPolicyResearch;Trusted_Connection=yes;")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

In [22]:
df_dividend_final["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
df_dividend_final["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
df_dividend_final

,Date,Dividends,Ticker,date_insert,time_insert
0,2006-11-01,1.471388,A,2023-05-07,01:56:54
1,2012-03-30,0.071531,A,2023-05-07,01:56:54
2,2012-06-29,0.071531,A,2023-05-07,01:56:54
3,2012-09-28,0.071531,A,2023-05-07,01:56:54
4,2012-12-27,0.071531,A,2023-05-07,01:56:54
...,...,...,...,...,...
46330,2022-04-20,0.325000,ZTS,2023-05-07,01:56:54
46331,2022-07-20,0.325000,ZTS,2023-05-07,01:56:54
46332,2022-10-31,0.325000,ZTS,2023-05-07,01:56:54
46333,2023-01-19,0.375000,ZTS,2023-05-07,01:56:54


In [23]:
df_stock_final["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
df_stock_final["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
df_stock_final

,city,state,zip,website,industry,sector,longBusinessSummary,marketCap,symbol,underlyingSymbol,shortName,longName,date_insert,time_insert
0,Santa Clara,CA,95051,https://www.agilent.com,Diagnostics & Research,Healthcare,"Agilent Technologies, Inc. provides application focused solutions to the life sciences, diagnostics, and applied chemical markets worldwide. The Life Sciences and Applied Markets segment offers liquid chromatography systems and components; liquid chromatography mass spectrometry systems; gas chromatography systems and components; gas chromatography mass spectrometry systems; inductively coupled plasma mass spectrometry instruments; atomic absorption instruments; microwave plasma-atomic emission spectrometry instruments; inductively coupled plasma optical emission spectrometry instruments; raman spectroscopy; cell analysis plate based assays; flow cytometer; real-time cell analyzer; cell imaging systems; microplate reader; laboratory software; information management and analytics; laboratory automation and robotic systems; dissolution testing; vacuum pumps, and measurement technologies. The Diagnostics and Genomics segment provides arrays for DNA mutation detection, genotyping, gene copy number determination, identification of gene rearrangements, DNA methylation profiling, gene expression profiling, next generation sequencing, target enrichment and genetic data management, and interpretation support software; and produces synthesized oligonucleotide. It also offers immunohistochemistry in situ hybridization, and hematoxylin and eosin staining and special staining; consumables, and software for quality control analysis of nucleic acid samples; and reagents for use in turbidimetry and flow cytometry, as well as develops liquid-based pharmacodiagnostics. The Agilent CrossLab segment provides GC and LC columns, sample preparation products, custom chemistries, and laboratory instrument supplies; and startup, operational, training, compliance support, software as a service, asset management, and consultation services. The company markets its products through direct sales, distributors, resellers, manufacturer's representatives, and electronic commerce. Agilent Technologies, Inc. was incorporated in 1999 and is headquartered in Santa Clara, California.",39417077760,A,A,"Agilent Technologies, Inc.","Agilent Technologies, Inc.",2023-05-07,01:57:25
1,Fort Worth,TX,76155,https://www.aa.com,Airlines,Industrials,"American Airlines Group Inc., through its subsidiaries, operates as a network air carrier. The company provides scheduled air transportation services for passengers and cargo through its hubs in Charlotte, Chicago, Dallas/Fort Worth, Los Angeles, Miami, New York, Philadelphia, Phoenix, and Washington, D.C., as well as through partner gateways in London, Doha, Madrid, Seattle/Tacoma, Sydney, and Tokyo. As of December 31, 2022, it operated a mainline fleet of 925 aircraft. The company was formerly known as AMR Corporation and changed its name to American Airlines Group Inc. in December 2013. American Airlines Group Inc. was founded in 1926 and is headquartered in Fort Worth, Texas.",9055209472,AAL,AAL,"American Airlines Group, Inc.",American Airlines Group Inc.,2023-05-07,01:57:25
2,Raleigh,NC,27609,https://www.advanceautoparts.com,Specialty Retail,Consumer Cyclical,"Advance Auto Parts, Inc. provides automotive replacement parts, accessories, batteries, and maintenance items for domestic and imported cars, vans, sport utility vehicles, and light and heavy duty trucks. The company offers battery accessories; belts and hoses; brakes and brake pads; chassis and climate control parts; clutches and drive shafts; engines and engine parts; exhaust systems and parts; hub assemblies; ignition components and wires; radiators and cooling parts; starters and alternators; and steering and alignment parts. It also offers air conditioning chemicals and accessories; air fresheners; antifreeze and washer fluids; electrical wires and f

In [27]:
df_SP500 = df_SP500.rename(columns={0: "ticker"})

In [28]:
df_SP500

,ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
498,YUM
499,ZBH
500,ZBRA
501,ZION


In [29]:
df_SP500["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
df_SP500["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
df_SP500

,ticker,date_insert,time_insert
0,A,2023-05-07,01:59:17
1,AAL,2023-05-07,01:59:17
2,AAP,2023-05-07,01:59:17
3,AAPL,2023-05-07,01:59:17
4,ABBV,2023-05-07,01:59:17
...,...,...,...
498,YUM,2023-05-07,01:59:17
499,ZBH,2023-05-07,01:59:17
500,ZBRA,2023-05-07,01:59:17
501,ZION,2023-05-07,01:59:17


In [31]:
df_SP500.to_sql('tb_sp500_tickers', schema='dbo', con = engine, if_exists='append',index=False)

In [33]:
result = engine.execute('SELECT COUNT(*) FROM [dbo].[tb_sp500_tickers]')
result.fetchall()[0][0] 

503

In [37]:
df_dividend_final.to_sql('tb_dividends', schema='dbo', con = engine, if_exists='append',index=False)

In [38]:
result = engine.execute('SELECT COUNT(*) FROM [dbo].[tb_dividends]')
result.fetchall()[0][0] 

46335

In [39]:
df_stock_final.to_sql('tb_company_info', schema='dbo', con = engine, if_exists='append',index=False)

In [40]:
result = engine.execute('SELECT COUNT(*) FROM [dbo].[tb_company_info]')
result.fetchall()[0][0] 

503